In [1]:
# This is the training script for fine-tuning bert on 
# unaltered GPT data and manually labelled data
# For better performance/generalization, look for augmented dataset
# Read README.md for comments and details.

In [2]:

# all classes

classes = ["banking","valuation","household","real estate","corporate","external","sovereign","technology", "climate", "energy", "health", "eu"]


In [3]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments
import torch
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support,top_k_accuracy_score
import math
import pickle
from datasets import Dataset

In [4]:
# load bert-based and finbert
model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(classes))
finbert = AutoModelForSequenceClassification.from_pretrained('ProsusAI/finbert')
tokenizer = AutoTokenizer.from_pretrained('ProsusAI/finbert', use_fast =True)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [5]:
# weights transfer for encoder layers only 
finbert_weights = finbert.state_dict()
model_weights = model.state_dict()
del finbert_weights["bert.pooler.dense.weight"]
del finbert_weights["bert.pooler.dense.bias"]
del finbert_weights["classifier.weight"]
del finbert_weights["classifier.bias"]
finbert_weights["bert.pooler.dense.weight"] = model_weights["bert.pooler.dense.weight"]
finbert_weights["bert.pooler.dense.bias"] = model_weights["bert.pooler.dense.bias"]
finbert_weights["classifier.weight"] = model_weights["classifier.weight"]
finbert_weights["classifier.bias"] = model_weights["classifier.bias"]

model.load_state_dict(finbert_weights)


<All keys matched successfully>

In [6]:
# flatten to one list for all 3

# manual labelled
with open('train_data.pickle', 'rb') as file:
    train = pickle.load(file)

# gpt labelled p1
with open('gpt.pickle', 'rb') as file:
    gpt = pickle.load(file)

# gpt labelled p2
with open('gpt_p2.pickle', 'rb') as file:
    gpt2 = pickle.load(file)
gpt = [item for sublist in gpt for item in sublist]
gpt2 = [item for sublist in gpt2 for item in sublist]

mixed = gpt + gpt2

In [7]:
print(len(mixed))

2458


In [8]:
sample = 1

additional_text = []
additional_label = []

for idx in range(1,len(mixed)-1):
    sent1, sent2 = mixed[idx-1:idx+1]
    for i in range(1):
        additional_text.append(sent1["text"]+' '+ sent2["text"])
        dist = (np.array(sent1["dist"]) + np.array(sent2["dist"]))/2
        additional_label.append(np.argmax(dist))

for idx in range(2,len(mixed)-1):
    sent1, sent2, sent3 = mixed[idx-2:idx+1]
    for i in range(1):
        additional_text.append(sent1["text"]+' '+ sent2["text"]+' '+sent3["text"])
        dist = (np.array(sent1["dist"]) + np.array(sent2["dist"])+ np.array(sent3["dist"]))/3
        additional_label.append(np.argmax(dist))

text_max = [item["text"] for i in range(sample) for item in train]
label_max = [np.argmax(item["dist"]) for i in range(sample) for item in train]

text_max_mixed = [item["text"] for i in range(sample) for item in mixed]
label_max_mixed = [np.argmax(item["dist"]) for i in range(sample) for item in mixed]


In [9]:
import random
def randomize(text, label):
    temp = list(zip(text, label))
    random.shuffle(temp)
    comb_text, comb_label =  zip(*temp)
    return comb_text, comb_label

# a1 = randomize(text_max, label_max)
a2 = randomize(text_max_mixed, label_max_mixed)
a3 = randomize(additional_text, additional_label)

# test_text, train_text =  [*a1[0][math.ceil(len(a1[0])*0.8):],*a2[0][math.ceil(len(a2[0])*0.8):],*a3[0][math.ceil(len(a3[0])*0.8):] ], [*a1[0][:math.ceil(len(a1[0])*0.8)],*a2[0][:math.ceil(len(a2[0])*0.8)],*a3[0][:math.ceil(len(a3[0])*0.8)] ]
# test_label, train_label = [*a1[1][math.ceil(len(a1[1])*0.8):],*a2[1][math.ceil(len(a2[1])*0.8):],*a3[1][math.ceil(len(a3[1])*0.8):] ],  [*a1[1][:math.ceil(len(a1[1])*0.8)],*a2[1][:math.ceil(len(a2[1])*0.8)],*a3[1][:math.ceil(len(a3[1])*0.8)] ]

test_text, train_text =  [*a2[0][math.ceil(len(a2[0])*0.8):],*a3[0][math.ceil(len(a3[0])*0.8):] ], [*a2[0][:math.ceil(len(a2[0])*0.8)],*a3[0][:math.ceil(len(a3[0])*0.8)] ]
test_label, train_label = [*a2[1][math.ceil(len(a2[1])*0.8):],*a3[1][math.ceil(len(a3[1])*0.8):] ],  [*a2[1][:math.ceil(len(a2[1])*0.8)],*a3[1][:math.ceil(len(a3[1])*0.8)] ]


In [10]:
print(len(train_text))
print(len(test_text))

5896
1473


In [11]:
assert len(train_text) == len(train_label)
assert len(test_text) == len(test_label)

In [12]:
train_dataset = Dataset.from_dict({"text":train_text, "label":train_label})
test_dataset = Dataset.from_dict({"text":test_text, "label":test_label})

In [13]:
def tokenize(batch):
    return tokenizer(batch['text'], padding=True, truncation=True)
train_dataset = train_dataset.map(tokenize, batched=True, batch_size=len(train_dataset))
test_dataset = test_dataset.map(tokenize, batched=True, batch_size=len(test_dataset))
train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

100%|██████████| 1/1 [00:00<00:00,  8.73ba/s]


In [14]:
train_dataset["input_ids"]

tensor([[ 101, 2429, 2000,  ...,    0,    0,    0],
        [ 101, 1998, 2057,  ...,    0,    0,    0],
        [ 101, 2004, 1010,  ...,    0,    0,    0],
        ...,
        [ 101, 2004, 1045,  ...,    0,    0,    0],
        [ 101, 1998, 2057,  ...,    0,    0,    0],
        [ 101, 2144, 2978,  ...,    0,    0,    0]])

In [15]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
    acc = accuracy_score(labels, preds)
    top3 = top_k_accuracy_score(labels, pred.predictions,k=3)
    top2 = top_k_accuracy_score(labels, pred.predictions,k=2)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall,
        'top3': top3,
        'top2': top2
    }

training_args = TrainingArguments(
    
    output_dir='./results',
    learning_rate=2e-5,
    num_train_epochs=20,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    metric_for_best_model="accuracy",
    evaluation_strategy='epoch',
    save_strategy = "epoch",
    logging_dir='./logs',
    save_total_limit = 1, # Only last 5 models are saved. Older ones are deleted.
    load_best_model_at_end=True,
)
   
trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

In [16]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running training *****
  Num examples = 5896
  Num Epochs = 20
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 7380
  5%|▍         | 368/7380 [02:52<45:09,  2.59it/s]The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 1473
  Batch size = 64
C:\Users\felix\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg

{'eval_loss': 1.2970434427261353, 'eval_accuracy': 0.603530210454854, 'eval_f1': 0.2705597875498683, 'eval_precision': 0.2994106020459237, 'eval_recall': 0.28403628556726984, 'eval_top3': 0.8553971486761711, 'eval_top2': 0.7834351663272233, 'eval_runtime': 10.1238, 'eval_samples_per_second': 145.498, 'eval_steps_per_second': 2.371, 'epoch': 1.0}


Model weights saved in ./results\checkpoint-369\pytorch_model.bin
Deleting older checkpoint [results\checkpoint-382] due to args.save_total_limit
  7%|▋         | 501/7380 [04:11<37:23,  3.07it/s]

{'loss': 1.7254, 'learning_rate': 2e-05, 'epoch': 1.36}


 10%|▉         | 737/7380 [06:02<1:37:00,  1.14it/s]The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 1473
  Batch size = 64
C:\Users\felix\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))

 10%|█         | 738/7380 [06:12<1:36:59,  1.14it/s]Saving model checkpoint to ./results\checkpoint-738
Configuration saved in ./results\checkpoint-738\config.json


...

Model weights saved in ./results\checkpoint-7011\pytorch_model.bin
Deleting older checkpoint [results\checkpoint-6642] due to args.save_total_limit
100%|█████████▉| 7379/7380 [31:00<00:00,  7.11it/s]The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 1473
  Batch size = 64

100%|██████████| 7380/7380 [31:03<00:00,  7.11it/s]Saving model checkpoint to ./results\checkpoint-7380
Configuration saved in ./results\checkpoint-7380\config.json


{'eval_loss': 1.3936309814453125, 'eval_accuracy': 0.8173794976238968, 'eval_f1': 0.6840348388474689, 'eval_precision': 0.6842618838537003, 'eval_recall': 0.6890858084435739, 'eval_top3': 0.9205702647657841, 'eval_top2': 0.8968092328581126, 'eval_runtime': 3.1057, 'eval_samples_per_second': 474.289, 'eval_steps_per_second': 7.728, 'epoch': 20.0}


Model weights saved in ./results\checkpoint-7380\pytorch_model.bin
Deleting older checkpoint [results\checkpoint-7011] due to args.save_total_limit


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from ./results\checkpoint-4059 (score: 0.8194161575016972).
100%|██████████| 7380/7380 [31:05<00:00,  3.96it/s]

{'train_runtime': 1865.3098, 'train_samples_per_second': 63.217, 'train_steps_per_second': 3.956, 'train_loss': 0.248817716664092, 'epoch': 20.0}


TrainOutput(global_step=7380, training_loss=0.248817716664092, metrics={'train_runtime': 1865.3098, 'train_samples_per_second': 63.217, 'train_steps_per_second': 3.956, 'train_loss': 0.248817716664092, 'epoch': 20.0})

In [17]:
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 1473
  Batch size = 64
100%|██████████| 24/24 [00:02<00:00,  8.09it/s]


{'eval_loss': 1.1953911781311035,
 'eval_accuracy': 0.8194161575016972,
 'eval_f1': 0.6611845014083525,
 'eval_precision': 0.6647863367726922,
 'eval_recall': 0.6648780821314514,
 'eval_top3': 0.9260013577732519,
 'eval_top2': 0.8988458927359131,
 'eval_runtime': 3.0967,
 'eval_samples_per_second': 475.668,
 'eval_steps_per_second': 7.75,
 'epoch': 20.0}

In [18]:

predict_dataset = Dataset.from_dict({"text":["In contrast to the radical forces buffeting valuations, for most companies, 2020 was a year of “strategy lockdown.",
"Domestic policies thus tended to reinforce negative spillovers and exacerbate systemic risk across the euro area.",
"Mortgage interest rate in selected European countries as of 4th quarter of 2019 and 2020 increased",
"Accordingly, I shall spend most of my allotted time outlining the chosen monetary policy instruments and procedures of the ESCB and the considerations that have been raised relating to strategy.",
"Credit card interest rate has gone up a lot."]})

def tokenize(batch):
    return tokenizer(batch['text'], padding=True, truncation=True)
predict_dataset = predict_dataset.map(tokenize, batched=True, batch_size=len(train_dataset))

trainer.predict(predict_dataset)

100%|██████████| 1/1 [00:00<00:00, 499.86ba/s]
The following columns in the test set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running Prediction *****
  Num examples = 5
  Batch size = 64
  0%|          | 0/1 [00:00<?, ?it/s]

PredictionOutput(predictions=array([[ 0.08580349,  0.3426192 , -1.3598568 ,  1.052115  ,  6.6663265 ,
        -0.5810849 , -1.6031407 , -1.384228  , -1.4713211 ,  0.7213736 ,
        -0.4400661 , -0.30740368],
       [-0.645603  , -0.08085273, -1.6559494 , -1.2694521 , -1.6604722 ,
        -0.727355  ,  9.290436  , -1.5795223 , -0.2217362 , -2.9336421 ,
        -1.8644572 , -0.713128  ],
       [ 1.1604235 , -2.2036273 ,  6.8688965 ,  3.2814012 , -1.3509673 ,
        -1.5115111 , -1.6118517 , -1.1785508 , -0.31068662, -1.4158335 ,
        -1.3538754 , -1.7547491 ],
       [ 0.07224072, -1.5954038 , -1.1340033 , -1.2084347 , -1.4745549 ,
        -0.59224206,  9.260652  , -1.2765049 , -0.20540814, -3.1987143 ,
        -1.6362734 , -1.3360586 ],
       [ 9.143754  ,  1.0539382 , -0.69552696, -0.89807874, -1.3315082 ,
        -1.131033  , -0.21255666, -0.88644826, -0.5899793 , -2.256294  ,
        -1.883272  , -2.8046684 ]], dtype=float32), label_ids=None, metrics={'test_runtime': 0.017, '